In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df_sms = pd.read_csv('data/smsspamcollection.tsv', sep='\t')
df_sms.head()

,label,message,length,punct
0,ham,"Go until jurong point, crazy.. Available only ...",111,9
1,ham,Ok lar... Joking wif u oni...,29,6
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155,6
3,ham,U dun say so early hor... U c already then say...,49,6
4,ham,"Nah I don't think he goes to usf, he lives aro...",61,2


In [3]:
df_sms.isnull().sum()

label      0
message    0
length     0
punct      0
dtype: int64

In [4]:
df_sms.shape

(5572, 4)

In [5]:
df_sms['label'].value_counts()

ham     4825
spam     747
Name: label, dtype: int64

In [14]:
df_sms['label'].nunique()

2

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
X = df_sms['message']
y = df_sms['label']

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [18]:
from sklearn.feature_extraction.text import CountVectorizer

In [19]:
cnt_vect = CountVectorizer()

In [20]:
X_train_cnts = cnt_vect.fit_transform(X_train)
X_train_cnts

<3900x7263 sparse matrix of type '<class 'numpy.int64'>'
	with 52150 stored elements in Compressed Sparse Row format>

In [21]:
X_train.shape

(3900,)

In [22]:
X_train_cnts.shape

(3900, 7263)

In [23]:
from sklearn.feature_extraction.text import TfidfTransformer

In [24]:
tfidf_trans = TfidfTransformer()

In [25]:
X_train_tfidf = tfidf_trans.fit_transform(X_train_cnts)
X_train_tfidf.shape

(3900, 7263)

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [27]:
tfidf_vect = TfidfVectorizer()

In [28]:
X_train_tfidf = tfidf_vect.fit_transform(X_train)

In [29]:
from sklearn.svm import LinearSVC

In [30]:
clf = LinearSVC()

In [31]:
clf.fit(X_train_tfidf, y_train)

LinearSVC()

In [32]:
from sklearn.pipeline import Pipeline

In [33]:
text_clf = Pipeline([('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])

In [34]:
text_clf.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])

In [35]:
y_pred = text_clf.predict(X_test)

In [36]:
from sklearn.metrics import confusion_matrix, classification_report

In [37]:
print(confusion_matrix(y_test, y_pred))

[[1445    3]
 [  10  214]]


In [38]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         ham       0.99      1.00      1.00      1448
        spam       0.99      0.96      0.97       224

    accuracy                           0.99      1672
   macro avg       0.99      0.98      0.98      1672
weighted avg       0.99      0.99      0.99      1672



In [39]:
from sklearn import metrics

In [40]:
metrics.accuracy_score(y_test, y_pred)

0.9922248803827751

In [41]:
text_clf.predict(["Congratulation! You have won and price"])

array(['spam'], dtype=object)

## Movie Review Dataset

In [42]:
df_reviews = pd.read_csv('data/moviereviews.tsv', sep='\t')
df_reviews.head()

,label,review
0,neg,how do films like mouse hunt get into theatres...
1,neg,some talented actresses are blessed with a dem...
2,pos,this has been an extraordinary year for austra...
3,pos,according to hollywood movies made in last few...
4,neg,my first press screening of 1998 and already i...


In [43]:
df_reviews.shape

(2000, 2)

In [47]:
df_reviews.isnull().sum()

label      0
review    35
dtype: int64

In [48]:
df_reviews.dropna(inplace=True)
df_reviews.isnull().sum()

label     0
review    0
dtype: int64

##### Dropping Blanks in Review

In [49]:
mystring = 'hello'
myempty = ' '
mystring.isspace(), myempty.isspace()

(False, True)

In [51]:
blank_idx = []

for idx, lb, rv in df_reviews.itertuples():
    if rv.isspace():
        blank_idx.append(idx)
        
blank_idx

[57,
 71,
 147,
 151,
 283,
 307,
 313,
 323,
 343,
 351,
 427,
 501,
 633,
 675,
 815,
 851,
 977,
 1079,
 1299,
 1455,
 1493,
 1525,
 1531,
 1763,
 1851,
 1905,
 1993]

In [52]:
df_reviews.drop(index=blank_idx, inplace=True)
df_reviews.shape

(1938, 2)

In [53]:
from sklearn.model_selection import train_test_split

In [54]:
X = df_reviews['review']
y = df_reviews['label']

In [55]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [56]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC

In [57]:
text_clf = Pipeline([
    ('tfidf', TfidfVectorizer()), 
    ('clf', LinearSVC())
])

In [58]:
text_clf.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])

In [59]:
y_pred = text_clf.predict(X_test)

In [60]:
print(confusion_matrix(y_test, y_pred))

[[235  47]
 [ 41 259]]


In [61]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         neg       0.85      0.83      0.84       282
         pos       0.85      0.86      0.85       300

    accuracy                           0.85       582
   macro avg       0.85      0.85      0.85       582
weighted avg       0.85      0.85      0.85       582

